In [18]:
! pip install -U spacy
! python -m spacy download en
! pip install spacymoji
! pip install emoji
! pip install gensim
! pip install sklearn


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 12.8/12.8 MB 3.9 MB/s eta 0:00:00
[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use
the full pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
import pandas as pd
data = pd.read_csv('corto.csv')

In [23]:
textos = []
for instancia in data.values:
    textos.append(instancia[1])

In [24]:
import spacy
import emoji

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("emoji", first=True)

textos_tokenizados =[]
for t in textos:
    t= emoji.demojize(t)
    doc=nlp(t)
    lexical_tokens = [token.lemma_.lower() for token in doc if len(token.text) > 3 and token.is_alpha]
    textos_tokenizados.append(lexical_tokens)


Anyone want to chat during school, I am bored, DM me or add me on discord UltraChad#8308. Anyone want to chat during school, I am bored, DM me or add me on discord UltraChad#8308.
Marble Blast Ultra Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia Nostalgia N

In [25]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from gensim.test.utils import get_tmpfile

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(textos_tokenizados)]
model = Doc2Vec(documents, vector_size=150, window=2, dm=1, epochs=100, workers=4)
# DM siempre a 1 para que sea distributed memory (que tenga en cuenta el contexto)
# Window se puede cambiar pero tener en cuenta que cuanto mayor sea mas coste genera
# Negative: siempre a 0 para que aprenda que palabras son ruido
#Los que se pueden probar para parametro ekorketa:
    # VECTOR SIZE!!!! (tamaño del embedding: dimensiones)
    # Epochs: numero de iteraciones por el dataset
    # Alpha: learning rate inicial
    # Min alpha: El valor de LR con el que va acabar → Igual no poner?
    # Max_vocab_size: Limita el tamaño maximo de las palabras a tener en cuenta (borra las menos frecuentes)

model.build_vocab(documents)
model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)

model.save(get_tmpfile("my_doc2vec_model"))

#vector = model.infer_vector(['wife', 'threaten', 'leave', 'wife', 'good', 'because', 'cheat', 'twice', 'lie', 'much', 'that', 'have', 'decide', 'refuse', 'back', 'day', 'begin', 'threaten', 'suicide', 'have', 'tirelessly', 'spend', 'these', 'paat', 'day', 'talk', 'keep'])

#similar_documents = model.dv.most_similar(vector) # Deberia salir 0 porque es el documento del que se han sacado todas las palabras

In [26]:
from sklearn.cluster import DBSCAN
import numpy as np

document_vectors = [model.infer_vector(doc) for doc in textos_tokenizados]

# Aplicar DBSCAN a los vectores de documentos
dbscan = DBSCAN(eps=2, min_samples=2, leaf_size=5)  # Ajusta los parámetros según tu caso
labels = dbscan.fit_predict(np.array(document_vectors))

# Los resultados del clustering están en 'labels'
print("Etiquetas de clusters:", labels)

Etiquetas de clusters: [-1  0  0 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1  0 -1 -1 -1 -1
  0  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1  0 -1  0 -1 -1
 -1 -1 -1  0  0 -1 -1  0 -1 -1 -1  0  0 -1]


In [27]:
np.array(document_vectors)

array([[-0.09631877,  0.04839203, -0.25529438, ...,  0.14251646,
         0.01366897, -0.14677592],
       [ 0.0489575 , -0.11862181,  0.02338012, ..., -0.01025372,
         0.00640117, -0.02973221],
       [-0.11108461, -0.17257743, -0.04706689, ...,  0.14581136,
         0.28358763, -0.12058074],
       ...,
       [-0.12342906, -0.44393882, -0.07893974, ..., -0.0218531 ,
         0.13121454, -0.02291693],
       [ 0.07600959,  0.03359744, -0.02560849, ...,  0.14621036,
        -0.02106139, -0.34631124],
       [ 0.83251196,  1.8498625 ,  0.80279034, ..., -2.6948671 ,
         1.1668283 , -1.2073256 ]], dtype=float32)